In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
import torchvision
from torchvision import transforms
import numpy as np
import cv2
from sklearn.cluster import KMeans
import pandas as pd
import requests
from PIL import Image
import io
import numpy as np
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights

In [3]:
# 클래스 정의
class CSVDataLoader:
    def __init__(self, url):
        """
        CSV 파일을 다운로드하고 데이터프레임으로 로드하는 클래스
        Args:
            url (str): 다운로드할 CSV 파일의 URL
        """
        self.url = url
        self.file_name = "data.csv"

    def download_csv(self):
        """
        CSV 파일 다운로드
        """
        try:
            response = requests.get(self.url)
            response.raise_for_status()  # HTTP 에러가 발생하면 예외 발생
            with open(self.file_name, "wb") as file:
                file.write(response.content)
            print(f"CSV 파일이 '{self.file_name}' 이름으로 저장되었습니다.")
        except requests.exceptions.RequestException as e:
            print(f"파일 다운로드 중 에러 발생: {e}")

    def load_data(self, sample_size=None):
        """
        CSV 데이터를 Pandas DataFrame으로 로드하고 샘플 데이터를 반환
        Args:
            sample_size (int): 반환할 샘플 데이터 크기 (None이면 전체 데이터)
        Returns:
            pd.DataFrame: 로드된 데이터프레임 또는 샘플 데이터
        """
        try:
            data = pd.read_csv(self.file_name)
            print(f"데이터가 성공적으로 로드되었습니다. (총 {len(data)}개 행)")
            if sample_size:
                data = data.sample(n=sample_size, random_state=42)
            # 인덱스 리셋
            data.reset_index(drop=True, inplace=True)
            return data
        except Exception as e:
            print(f"데이터 로드 중 에러 발생: {e}")
            return pd.DataFrame()


    def url_to_pixels(self, url):
        """
        이미지 URL을 받아서 픽셀 값으로 변환
        Args:
            url (str): 이미지 URL
        Returns:
            np.ndarray: 이미지의 RGB 픽셀 값
        """
        try:
            response = requests.get(url)
            response.raise_for_status()
            image = Image.open(io.BytesIO(response.content))
            # 이미지를 RGB로 변환
            image = image.convert('RGB')
            return np.array(image)
        except requests.exceptions.RequestException as e:
            print(f"이미지 다운로드 중 에러 발생: {e}")
            return None

    def convert_images_in_dataframe(self, df, image_column):
        """
        데이터프레임의 이미지 URL 칼럼을 이미지 픽셀 값으로 변환
        Args:
            df (pd.DataFrame): 데이터프레임
            image_column (str): 이미지 URL이 들어있는 컬럼 이름
        Returns:
            pd.DataFrame: 이미지 URL을 픽셀 값으로 변환한 데이터프레임
        """
        df[image_column] = df[image_column].apply(lambda url: self.url_to_pixels(url) if pd.notna(url) else None)
        return df

class MaskRCNNProcessor:
    def __init__(self, device=None):
        """
        Mask R-CNN 모델 초기화 및 설정
        Args:
            device (str): 사용할 디바이스 (cuda 또는 cpu)
        """
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self._get_mask_rcnn_model()

    def _get_mask_rcnn_model(self):
        """
        Mask R-CNN 모델 로드
        Returns:
        model: Mask R-CNN 모델
        """
        weights = MaskRCNN_ResNet50_FPN_Weights.COCO_V1  # COCO 데이터셋의 학습된 가중치를 사용
        model = maskrcnn_resnet50_fpn(weights=weights)
        model.eval()
        model.to(self.device)
        return model


    def preprocess_image(self, image_array):
        """
        이미지 전처리 함수
        Args:
            image_array (ndarray): 원본 이미지 배열
        Returns:
            torch.Tensor: 전처리된 이미지 텐서
        """
        transform = transforms.Compose([
            transforms.ToTensor(),
        ])
        image_tensor = transform(image_array).unsqueeze(0).to(self.device)
        return image_tensor

    def get_masks(self, image_tensor):
        """
        Mask R-CNN으로 객체 분할 수행
        Args:
            image_tensor (torch.Tensor): 전처리된 이미지 텐서
        Returns:
            dict: 객체 분할 결과
        """
        with torch.no_grad():
            predictions = self.model(image_tensor)
        return predictions

class ColorProcessor:
    def __init__(self, num_colors=1):
        """
        색상 처리 클래스 초기화
        Args:
            num_colors (int): 추출할 색상의 개수
        """
        self.num_colors = num_colors

    def generate_initial_mask(self, mask):
        """
        초기 마스크 생성
        Args:
            mask (torch.Tensor or ndarray): 분할 마스크
        Returns:
            ndarray: 초기 마스크
        """
        mask_np = mask.squeeze().cpu().numpy() if isinstance(mask, torch.Tensor) else mask.squeeze()
        thresholded_mask = (mask_np > 0.5).astype(np.uint8) * 255  # 0.5 이상의 값은 흰색, 나머지 영역은 검은색
        return thresholded_mask

    def extract_dominant_color(self, image, mask, num_colors=1):
        """
        마스크를 사용하여 이미지에서 대표 색상 추출
        Args:
            image (ndarray): 원본 이미지
            mask (ndarray): 마스크 이미지
            num_colors (int): 추출할 색상의 개수 (기본값 3)
        Returns:
            list: 대표 색상 리스트
            list: 각 색상의 비율 리스트
        """
        masked_image = cv2.bitwise_and(image, image, mask=mask)  # 마스크가 적용된 이미지
        pixels = masked_image[mask > 0].reshape(-1, 3)  # mask가 0이 아닌 부분만 고려

        # 유효한 픽셀만 사용 (너무 어두운 픽셀 제외)
        pixels = pixels[np.any(pixels > 10, axis=1)]

        # KMeans 클러스터링을 통해 대표 색상 추출
        if len(pixels) < num_colors:
            return [np.mean(pixels, axis=0)], [1.0]

        kmeans = KMeans(n_clusters=num_colors, random_state=0)
        kmeans.fit(pixels)

        # 각 클러스터가 차지하는 비율 계산
        unique, counts = np.unique(kmeans.labels_, return_counts=True)
        total_pixels = len(pixels)
        color_ratios = counts / total_pixels

        return kmeans.cluster_centers_, color_ratios

    def generate_heatmap(self, image, initial_mask):
        """
        히트맵 생성
        Args:
            image (ndarray): 원본 이미지
            initial_mask (ndarray): 초기 마스크
        Returns:
            ndarray: 히트맵 이미지
        """
        heatmap = cv2.applyColorMap(initial_mask, cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255.0
        image = np.float32(image) / 255.0
        superimposed_image = cv2.addWeighted(image, 0.7, heatmap, 0.3, 0)
        return superimposed_image


class DatasetProcessor:
    def __init__(self, mask_rcnn_processor, color_processor):
        """
        데이터셋 처리 클래스 초기화
        Args:
            mask_rcnn_processor (MaskRCNNProcessor): Mask R-CNN 처리 객체
            color_processor (ColorProcessor): 색상 처리 객체
        """
        self.mask_rcnn_processor = mask_rcnn_processor
        self.color_processor = color_processor

    def process_image(self, image_array):
        """
        이미지를 처리하여 대표 색상, 비율, 마스크 및 히트맵을 생성
        Args:
            image_array (ndarray): 원본 이미지 배열
        Returns:
            tuple: 초기 마스크, 대표 색상, 비율, 히트맵
        """
        try:
            input_tensor = self.mask_rcnn_processor.preprocess_image(image_array)

            # Mask R-CNN을 사용하여 객체 분할
            predictions = self.mask_rcnn_processor.get_masks(input_tensor)
            masks = predictions[0]['masks']
            scores = predictions[0]['scores']
            labels = predictions[0]['labels']

            mask_np = None

            # 특정 객체 (예: 옷) 찾기
            for mask, score, label in zip(masks, scores, labels):
                if label.item() == 1:  # 옷을 포함하는 클래스의 인덱스 (예: COCO 데이터셋에서는 1이 사람으로 되어 있음)
                    mask_np = mask.squeeze().cpu().numpy()
                    break

            # 객체가 없으면 None 반환
            if mask_np is None:
                return None

            # 초기 마스크 생성
            initial_mask = self.color_processor.generate_initial_mask(mask_np)

            # 마스크만을 사용하여 대표 색상 및 비율 추출
            dominant_colors, color_ratios = self.color_processor.extract_dominant_color(image_array, initial_mask, num_colors=3)

            # 히트맵 생성
            heatmap_image = self.color_processor.generate_heatmap(image_array, initial_mask)
            return initial_mask, dominant_colors, color_ratios, heatmap_image

        except Exception as e:
            print(f"Error processing image: {e}")
            return None

    def label_personal_color(self, rgb):
        """
        주어진 RGB 값을 기반으로 퍼스널컬러를 라벨링
        Args:
            rgb (list or np.array): [R, G, B] 값 (0~255)
        Returns:
            str: 퍼스널컬러 라벨 (Spring Warm, Summer Cool, Autumn Warm, Winter Cool)
        """
        r, g, b = rgb

        # 색조: 따뜻한 색(웜) 또는 차가운 색(쿨) 판단
        if r >= g and r >= b:  # Red가 dominant
            tone = 'Warm'
        else:  # Blue or Green dominant
            tone = 'Cool'

        # 밝기: 밝음/어두움 판단 (YIQ 모델의 Y값 활용)
        brightness = 0.299 * r + 0.587 * g + 0.114 * b
        if brightness > 128:  # 밝으면
            lightness = 'Bright'
        else:  # 어두우면
            lightness = 'Dark'

        # 최종 라벨 결정
        if lightness == 'Bright' and tone == 'Warm':
            return 'Spring Warm'
        elif lightness == 'Bright' and tone == 'Cool':
            return 'Summer Cool'
        elif lightness == 'Dark' and tone == 'Warm':
            return 'Autumn Warm'
        else:
            return 'Winter Cool'

    def add_personal_color_labels(self, dataset):
        """
        데이터셋에 퍼스널컬러 라벨 추가
        Args:
            dataset (pd.DataFrame): 대표 색상이 포함된 데이터프레임
        Returns:
            pd.DataFrame: 퍼스널컬러 라벨이 추가된 데이터프레임
        """
        # 퍼스널컬러 라벨 추가
        dataset['personal_color'] = dataset['dominant_color'].apply(self.label_personal_color)
        return dataset

    def add_dominant_color_to_dataset(self, dataset):
        """
        데이터셋의 각 이미지에 대해 대표 색상 및 비율을 추가
        Args:
            dataset (pd.DataFrame): 이미지 데이터를 포함한 데이터프레임
        Returns:
            pd.DataFrame: 대표 색상 및 비율이 추가된 데이터프레임
        """
        dominant_color_list = []
        dominant_ratio_list = []
        valid_indices = []

        for idx, row in dataset.iterrows():
            image_array = row['이미지링크']  # 데이터프레임에서 이미지 데이터를 가져옴

            # 대표 색상 및 비율 추출
            result = self.process_image(image_array)

            # 객체가 감지되지 않으면 건너뛰기
            if result is None:
                continue

            _, dominant_colors, color_ratios, _ = result

            # 가장 높은 비율의 색상과 그 비율 선택
            max_ratio_idx = np.argmax(color_ratios)
            dominant_color = dominant_colors[max_ratio_idx]
            dominant_ratio = color_ratios[max_ratio_idx]

            # 유효한 값만 추가
            valid_indices.append(idx)
            dominant_color_list.append(dominant_color.astype(int).tolist())
            dominant_ratio_list.append(float(dominant_ratio))

        # 유효한 인덱스를 기반으로 데이터프레임 업데이트
        if valid_indices:
            dataset = dataset.loc[valid_indices].copy()
            dataset['dominant_color'] = dominant_color_list
            dataset['dominant_ratio'] = dominant_ratio_list
        else:
            print("No valid objects detected in the dataset.")
            dataset['dominant_color'] = []
            dataset['dominant_ratio'] = []

        # 퍼스널컬러 라벨 추가
        dataset = self.add_personal_color_labels(dataset)

        return dataset

# 깃허브 데이터 불러오기

In [6]:

if __name__ == "__main__":
    # 사용자 입력
    url = 'https://raw.githubusercontent.com/kieunseo/Fashion_x/refs/heads/main/data/file/clothes.csv?token=GHSAT0AAAAAAC5FDN7QFYUIDARLZT3YIT7WZ4JXSPQ'
    sample_size = 3000
    image_column = '이미지링크'
    # 클래스 사용
    loader = CSVDataLoader(url)
    loader.download_csv()

    # 데이터 로드 및 샘플링
    dataset = loader.load_data(sample_size=sample_size if sample_size > 0 else None)

    # 이미지 URL을 픽셀 값으로 변환
    dataset = loader.convert_images_in_dataframe(dataset, image_column)

    # 결과 출력
    print("로딩된 데이터셋:")
    print(dataset.head())

파일 다운로드 중 에러 발생: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/kieunseo/Fashion_x/refs/heads/main/data/file/clothes.csv?token=GHSAT0AAAAAAC5FDN7QFYUIDARLZT3YIT7WZ4JXSPQ
데이터가 성공적으로 로드되었습니다. (총 7120개 행)
이미지 다운로드 중 에러 발생: 404 Client Error: Not Found for url: https://image.msscdn.net/thumbnails/images/goods_img/20240428/4092312/4092312_17322578812645_big.jpg?w=780
이미지 다운로드 중 에러 발생: 404 Client Error: Not Found for url: https://image.msscdn.net/thumbnails/images/goods_img/20240719/4261057/4261057_17325227511671_big.jpg?w=780
이미지 다운로드 중 에러 발생: 404 Client Error: Not Found for url: https://image.msscdn.net/thumbnails/images/goods_img/20240620/4209355/4209355_17308602073825_big.jpg?w=780
이미지 다운로드 중 에러 발생: 404 Client Error: Not Found for url: https://image.msscdn.net/thumbnails/images/goods_img/20241022/4542610/4542610_17297325883588_big.jpg?w=780
이미지 다운로드 중 에러 발생: 404 Client Error: Not Found for url: https://image.msscdn.net/thumbnails/images/goods_img/20241107/4595116/4

In [7]:
dataset.shape

(3000, 7)

In [8]:
data = pd.read_csv("data_original.csv")
data.head()

,대분류,소분류,성별,계절,브랜드명,상품명,이미지링크
0,상의,맨투맨/스웨트,남성,봄,엠더바스,reach out 오버핏 스웨트셔츠 -메란지그레이,https://image.msscdn.net/thumbnails/images/goo...
1,상의,맨투맨/스웨트,남성,봄,넷소셜클럽,SMALL CURLY LOGO MTM (BLACK),https://image.msscdn.net/thumbnails/images/goo...
2,상의,맨투맨/스웨트,남성,봄,세인트크루,체스트 화이트캣 스웨트셔츠 CCS-809 블랙,https://image.msscdn.net/thumbnails/images/goo...
3,상의,맨투맨/스웨트,남성,봄,세인트크루,체스트 블랙아치 스웨트셔츠 CCS-809 그레이,https://image.msscdn.net/thumbnails/images/goo...
4,상의,맨투맨/스웨트,남성,봄,지크립,NEW YORKLIP 특양면 맨투맨,https://image.msscdn.net/thumbnails/images/goo...


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7120 entries, 0 to 7119
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   대분류     7120 non-null   object
 1   소분류     7120 non-null   object
 2   성별      7120 non-null   object
 3   계절      6789 non-null   object
 4   브랜드명    7120 non-null   object
 5   상품명     7120 non-null   object
 6   이미지링크   7120 non-null   object
dtypes: object(7)
memory usage: 389.5+ KB


In [10]:
if __name__ == "__main__":
    # 클래스 초기화
    device = "cuda" if torch.cuda.is_available() else "cpu"
    mask_rcnn_processor = MaskRCNNProcessor(device)
    color_processor = ColorProcessor(num_colors=1)
    dataset_processor = DatasetProcessor(mask_rcnn_processor, color_processor)

    # 데이터셋 처리
    processed_dataset = dataset_processor.add_dominant_color_to_dataset(dataset)

    # 결과 출력
    print(processed_dataset)

    # 각 행의 결과 시각화 (예시)
    for idx, row in processed_dataset.iterrows():
        print(f"이미지 {idx}: Dominant Color: {row['dominant_color']}, Ratio: {row['dominant_ratio']}")

Error processing image: pic should be PIL Image or ndarray. Got <class 'NoneType'>
Error processing image: pic should be PIL Image or ndarray. Got <class 'NoneType'>
Error processing image: pic should be PIL Image or ndarray. Got <class 'NoneType'>
Error processing image: pic should be PIL Image or ndarray. Got <class 'NoneType'>
Error processing image: pic should be PIL Image or ndarray. Got <class 'NoneType'>
Error processing image: pic should be PIL Image or ndarray. Got <class 'NoneType'>
Error processing image: pic should be PIL Image or ndarray. Got <class 'NoneType'>
Error processing image: pic should be PIL Image or ndarray. Got <class 'NoneType'>
Error processing image: pic should be PIL Image or ndarray. Got <class 'NoneType'>
Error processing image: pic should be PIL Image or ndarray. Got <class 'NoneType'>
Error processing image: pic should be PIL Image or ndarray. Got <class 'NoneType'>
Error processing image: pic should be PIL Image or ndarray. Got <class 'NoneType'>
Erro

- 데이터 병합

In [11]:
data_link = data[["브랜드명", "상품명", "이미지링크"]]
data_link.head()

,브랜드명,상품명,이미지링크
0,엠더바스,reach out 오버핏 스웨트셔츠 -메란지그레이,https://image.msscdn.net/thumbnails/images/goo...
1,넷소셜클럽,SMALL CURLY LOGO MTM (BLACK),https://image.msscdn.net/thumbnails/images/goo...
2,세인트크루,체스트 화이트캣 스웨트셔츠 CCS-809 블랙,https://image.msscdn.net/thumbnails/images/goo...
3,세인트크루,체스트 블랙아치 스웨트셔츠 CCS-809 그레이,https://image.msscdn.net/thumbnails/images/goo...
4,지크립,NEW YORKLIP 특양면 맨투맨,https://image.msscdn.net/thumbnails/images/goo...


In [12]:
merged_df = pd.merge(processed_dataset, data_link, on=["브랜드명", "상품명"], how="left")
print(merged_df.head())

   대분류         소분류  성별   계절    브랜드명                            상품명  \
0   바지  트레이닝/조거 팬츠  남성   여름  엔피스튜디오               코튼 와이드 스웨트 팬츠 블랙   
1   상의     셔츠/블라우스  남성    봄  필루미네이트  [유튜버 니들 착용] 오버핏 포레스트 체크 셔츠-블랙   
2  아우터       무스탕/퍼  남성  NaN     엘무드   A2 에비에이터 카우 하이드 무스탕 자켓 탠 브라운   
3  아우터     플리스/뽀글이  여성  NaN    피도디도        Fleece Zip Hoodie Black   
4  스커트       미니스커트  여성   가을  로맨틱크라운              샤이닝 로고 플리츠 스커트_블랙   

                                             이미지링크_x   dominant_color  \
0  [[[227, 228, 222], [227, 228, 222], [227, 228,...     [27, 31, 33]   
1  [[[255, 255, 255], [255, 255, 255], [255, 255,...     [35, 35, 35]   
2  [[[90, 89, 85], [87, 86, 82], [87, 86, 82], [8...     [41, 33, 29]   
3  [[[241, 241, 241], [241, 241, 241], [241, 241,...     [21, 22, 27]   
4  [[[206, 200, 204], [206, 200, 204], [205, 199,...  [169, 136, 110]   

   dominant_ratio personal_color  \
0        0.924865    Winter Cool   
1        0.592077    Autumn Warm   
2        0.522527    Autumn Warm

In [13]:
merged_df.columns

Index(['대분류', '소분류', '성별', '계절', '브랜드명', '상품명', '이미지링크_x', 'dominant_color',
       'dominant_ratio', 'personal_color', '이미지링크_y'],
      dtype='object')

In [14]:
result = merged_df.drop(columns=['이미지링크_x', 'dominant_color', 'dominant_ratio'], axis=1)
result.head()

,대분류,소분류,성별,계절,브랜드명,상품명,personal_color,이미지링크_y
0,바지,트레이닝/조거 팬츠,남성,여름,엔피스튜디오,코튼 와이드 스웨트 팬츠 블랙,Winter Cool,https://image.msscdn.net/thumbnails/images/goo...
1,상의,셔츠/블라우스,남성,봄,필루미네이트,[유튜버 니들 착용] 오버핏 포레스트 체크 셔츠-블랙,Autumn Warm,https://image.msscdn.net/thumbnails/images/goo...
2,아우터,무스탕/퍼,남성,NaN,엘무드,A2 에비에이터 카우 하이드 무스탕 자켓 탠 브라운,Autumn Warm,https://image.msscdn.net/thumbnails/images/goo...
3,아우터,플리스/뽀글이,여성,NaN,피도디도,Fleece Zip Hoodie Black,Winter Cool,https://image.msscdn.net/thumbnails/images/goo...
4,스커트,미니스커트,여성,가을,로맨틱크라운,샤이닝 로고 플리츠 스커트_블랙,Spring Warm,https://image.msscdn.net/thumbnails/images/goo...


In [15]:
result = result.rename(columns={'이미지링크_y':'이미지링크'})
result.head()

,대분류,소분류,성별,계절,브랜드명,상품명,personal_color,이미지링크
0,바지,트레이닝/조거 팬츠,남성,여름,엔피스튜디오,코튼 와이드 스웨트 팬츠 블랙,Winter Cool,https://image.msscdn.net/thumbnails/images/goo...
1,상의,셔츠/블라우스,남성,봄,필루미네이트,[유튜버 니들 착용] 오버핏 포레스트 체크 셔츠-블랙,Autumn Warm,https://image.msscdn.net/thumbnails/images/goo...
2,아우터,무스탕/퍼,남성,NaN,엘무드,A2 에비에이터 카우 하이드 무스탕 자켓 탠 브라운,Autumn Warm,https://image.msscdn.net/thumbnails/images/goo...
3,아우터,플리스/뽀글이,여성,NaN,피도디도,Fleece Zip Hoodie Black,Winter Cool,https://image.msscdn.net/thumbnails/images/goo...
4,스커트,미니스커트,여성,가을,로맨틱크라운,샤이닝 로고 플리츠 스커트_블랙,Spring Warm,https://image.msscdn.net/thumbnails/images/goo...


In [16]:
result.to_csv('fashion_data.csv', index=False)